In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

mercedes_df = pd.read_csv('Mercedes.csv')

# remove output column - y
y = mercedes_df['y']

# drop the output column from the dataframe
mercedes_df.drop(columns='y',inplace= True)


In [3]:
mercedes_df.shape
mercedes_df.describe()
# count the number of NaN values in each column
print(mercedes_df.isnull().sum())
# outcome: No missing values.

ID      0
X0      0
X1      0
X2      0
X3      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 377, dtype: int64


In [4]:
mercedes_df['X11'].nunique()

1

In [5]:
# check for unique values.
cols_to_drop = [];
for col in mercedes_df.columns:
    if(len(mercedes_df[col].unique()) == 1):
        print('Unique == 1: ',col)
        mercedes_df.drop(columns=col,inplace= True)
    elif(len(mercedes_df[col].unique()) == mercedes_df.shape[0]):
        print("Unique == Shape[0]",col)
        mercedes_df.drop(columns=col,inplace= True)     

# OUTCOME: removed all the unique value COLUMNS.

Unique == Shape[0] ID
Unique == 1:  X11
Unique == 1:  X93
Unique == 1:  X107
Unique == 1:  X233
Unique == 1:  X235
Unique == 1:  X268
Unique == 1:  X289
Unique == 1:  X290
Unique == 1:  X293
Unique == 1:  X297
Unique == 1:  X330
Unique == 1:  X347


In [6]:
print('New shape is: ',mercedes_df.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mercedes_df, y, test_size=0.20, random_state=0)

New shape is:  (4209, 364)


In [7]:
print(X_train.dtypes)
X_train['X0'].value_counts()

X0      object
X1      object
X2      object
X3      object
X4      object
         ...  
X380     int64
X382     int64
X383     int64
X384     int64
X385     int64
Length: 364, dtype: object


z     290
ak    272
y     267
t     246
ay    242
x     242
o     215
f     187
n     152
az    147
w     144
j     134
aj    121
s      85
ap     79
d      66
h      60
al     54
v      32
e      29
ai     27
af     26
m      25
ba     23
i      17
ax     16
a      16
am     16
at     15
l      14
aw     14
ad     13
aq     12
b      11
u      11
au      9
k       8
r       7
as      7
bc      6
c       3
q       2
ao      2
g       1
ab      1
aa      1
Name: X0, dtype: int64

In [8]:
# find out the object column names
for col in X_train.columns:
    if(X_train[col].dtype =='object'):
        print('object Column Name is: ', col)

object Column Name is:  X0
object Column Name is:  X1
object Column Name is:  X2
object Column Name is:  X3
object Column Name is:  X4
object Column Name is:  X5
object Column Name is:  X6
object Column Name is:  X8


In [12]:
from sklearn.preprocessing import LabelEncoder
lb_encoder = LabelEncoder()

for col in X_train.columns:
    if(X_train[col].dtype =='object'):
        X_train[col+'_lb_code'] = lb_encoder.fit_transform(X_train[col])
        print(X_train[[col, col+'_lb_code']].head(10))

# OUTCOME:  current column and label encoded column

In [10]:

for col in X_train.columns:
    if(X_train[col].dtype =='object'):
        X_train.drop(columns=col,inplace= True)

X_train.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


X10           int64
X12           int64
X13           int64
X14           int64
X15           int64
              ...  
X3_lb_code    int32
X4_lb_code    int32
X5_lb_code    int32
X6_lb_code    int32
X8_lb_code    int32
Length: 364, dtype: object

In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['Column Name'] = X_train.columns
vif

,VIF Factor,Column Name
0,inf,X10
1,inf,X12
2,inf,X13
3,inf,X14
4,inf,X15
...,...,...
359,2.886710,X3_lb_code
360,1.047781,X4_lb_code
361,1.246189,X5_lb_code
362,1.300657,X6_lb_code


In [14]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)


In [15]:
le_ext = LabelEncoderExt()
X1_train, X1_test, y1_train, y1_test = train_test_split(mercedes_df, y, test_size=0.20, random_state=0)

cat_cols = [col for col in X1_train.columns if X1_train[col].dtype == 'object']
print(cat_cols)
for col in cat_cols:
    le_ext.fit(X1_train[col])
    X1_train[col]= le_ext.transform(X1_train[col])
    X1_test[col] = le_ext.transform(X1_test[col])

X1_train.values

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


array([[ 7, 14, 16, ...,  0,  0,  0],
       [ 8, 24, 38, ...,  0,  0,  0],
       [29,  1,  9, ...,  0,  0,  0],
       ...,
       [40,  4, 16, ...,  0,  0,  0],
       [43, 21, 16, ...,  0,  0,  0],
       [ 1, 24, 31, ...,  0,  0,  0]], dtype=int64)

In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(X1_train.values, i) for i in range(X1_train.shape[1])]
vif1['Column Name'] = X1_train.columns
vif1

,VIF Factor,Column Name
0,2.682949,X0
1,6.617434,X1
2,20.745911,X2
3,2.886710,X3
4,1.047781,X4
...,...,...
359,inf,X380
360,inf,X382
361,1.903061,X383
362,2.214770,X384


In [17]:
# implement standard scaling before PCA 
# no need of doing scaling as data is 1 or 0
from sklearn.decomposition import PCA

pca = PCA()
principal_components = pca.fit_transform(X1_train)
pca.explained_variance_ratio_

array([3.81606205e-01, 2.08343380e-01, 1.35233418e-01, 1.21892677e-01,
       9.22270692e-02, 1.62112945e-02, 7.59262169e-03, 4.40510246e-03,
       2.94179497e-03, 2.46608100e-03, 2.36902522e-03, 2.11489299e-03,
       1.70314060e-03, 1.50796457e-03, 1.31069845e-03, 1.05452661e-03,
       9.77493984e-04, 8.92871865e-04, 7.89311374e-04, 7.20426439e-04,
       6.52971814e-04, 6.30446558e-04, 5.96737499e-04, 5.11711805e-04,
       4.90356263e-04, 4.35265987e-04, 4.25981490e-04, 3.85445584e-04,
       3.80593997e-04, 3.56817716e-04, 3.45186447e-04, 3.18877304e-04,
       3.03843915e-04, 2.85811808e-04, 2.67833961e-04, 2.49451607e-04,
       2.20918279e-04, 2.10209497e-04, 2.08222981e-04, 1.99562261e-04,
       1.91164684e-04, 1.82356860e-04, 1.77809608e-04, 1.74502069e-04,
       1.72116479e-04, 1.56965786e-04, 1.48601188e-04, 1.45759490e-04,
       1.41188306e-04, 1.37259615e-04, 1.33722923e-04, 1.28272427e-04,
       1.23621396e-04, 1.18063064e-04, 1.14059986e-04, 1.10538233e-04,
      

In [18]:
exp_var_df = pd.DataFrame({'Coeff.': np.arange(0, len(X1_train.columns)),
                           'Exp. Var': pca.explained_variance_ratio_,
                           'Cum. Exp. Var.': np.cumsum(pca.explained_variance_ratio_)  
})

exp_var_df.head(50)

,Coeff.,Exp. Var,Cum. Exp. Var.
0,0,0.381606,0.381606
1,1,0.208343,0.589950
2,2,0.135233,0.725183
3,3,0.121893,0.847076
4,4,0.092227,0.939303
5,5,0.016211,0.955514
6,6,0.007593,0.963107
7,7,0.004405,0.967512
8,8,0.002942,0.970454
9,9,0.002466,0.972920


In [19]:
pcafinal = PCA(n_components=30)
pca_train = pcafinal.fit_transform(X1_train)
X1_train_pca = pd.DataFrame(pca_train)

pca_test = pcafinal.transform(X1_test)
X1_test_pca = pd.DataFrame(pca_test)

In [20]:
pca_train.shape

(3367, 30)

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linreg_pca = LinearRegression()
linreg_pca.fit(X1_train_pca, y1_train)
linreg_pca_pred = linreg_pca.predict(X1_test_pca)

print('PCA Linear Regression MSE: %.2f' %mean_squared_error(y1_test, linreg_pca_pred))

PCA Linear Regression MSE: 109.59


In [22]:
linreg = LinearRegression()
linreg.fit(X1_train, y1_train)
linreg_pred = linreg.predict(X1_test)

print('Simple Liner Regression MSE: %.2e' %mean_squared_error(y1_test, linreg_pred))


Simple Liner Regression MSE: 2.13e+19


In [41]:
from sklearn.ensemble import RandomForestRegressor

rfreg_pca = RandomForestRegressor(random_state=2)
rfreg_pca.fit(X1_train_pca, y1_train)
rfreg_pca_pred = rfreg_pca.predict(X1_test_pca)

print('PCA Data RF MSE: %.2f' %mean_squared_error(y1_test, rfreg_pca_pred))

C:\Users\madhunagakishore.k\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


PCA Data RF MSE: 122.26


In [44]:
from sklearn.linear_model import Lasso

pca_lassoreg = Lasso()
pca_lassoreg.fit(X1_train_pca, y1_train)
pca_lassoreg.coef_

array([ 0.15337232,  0.01852314, -0.08072477, -0.0750485 ,  0.00609287,
        0.        ,  0.86447215, -2.22583746, -0.        ,  2.37671718,
        0.61797119, -2.27790805,  0.14234852, -1.38801567,  0.        ,
        0.87058399, -3.63193279,  0.        , -0.        ,  0.        ,
       -0.        ,  0.        ,  0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [48]:
np.nonzero(pca_lassoreg.coef_)

(array([ 0,  1,  2,  3,  4,  6,  7,  9, 10, 11, 12, 13, 15, 16],
       dtype=int64),)

In [49]:
pca_lassoreg_pred = pca_lassoreg.predict(X1_test_pca)
print('PCA Lasso MSE: %.2f' %mean_squared_error(y1_test, pca_lassoreg_pred)) 

PCA Lasso MSE: 117.28


In [52]:
pca_lassoreg.score

<bound method RegressorMixin.score of Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)>

In [53]:
from sklearn.metrics import r2_score

pca_lasso_r2 = r2_score(y1_test, pca_lassoreg_pred)
pca_lasso_r2

0.36722560057545806

In [54]:
print(r2_score(y1_test, rfreg_pca_pred))

0.3403577233664088


In [55]:
print(r2_score(y1_test, linreg_pca_pred))

0.4086899366318274


In [24]:
import statsmodels.api as sm
x=sm.add_constant(X_train)
model = sm.OLS(y_train,x).fit
model.summary()

AttributeError: 'function' object has no attribute 'summary'